<a href="https://colab.research.google.com/github/arnabbarui5/Movie-Revew-Classification-Tensor-Flow-Hub/blob/master/Movie_Review_Classification_TensorFlow_Hub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook classifies movie reviews as positive or negative using the text of the review. This is an example of binary—or two-class—classification, an important and widely applicable kind of machine learning problem.

The tutorial demonstrates the basic application of transfer learning with TensorFlow Hub and Keras.

We'll use the IMDB dataset that contains the text of 50,000 movie reviews from the Internet Movie Database. These are split into 25,000 reviews for training and 25,000 reviews for testing. The training and testing sets are balanced, meaning they contain an equal number of positive and negative reviews.

This notebook uses tf.keras, a high-level API to build and train models in TensorFlow, and TensorFlow Hub, a library and platform for transfer learning.

In [0]:
%tensorflow_version 2.x

import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub

In [2]:
tf.__version__

'2.2.0-rc1'

## TensorFlow Datasets Lists


In [3]:
tfds.list_builders()

['abstract_reasoning',
 'aeslc',
 'aflw2k3d',
 'amazon_us_reviews',
 'arc',
 'bair_robot_pushing_small',
 'beans',
 'big_patent',
 'bigearthnet',
 'billsum',
 'binarized_mnist',
 'binary_alpha_digits',
 'c4',
 'caltech101',
 'caltech_birds2010',
 'caltech_birds2011',
 'cars196',
 'cassava',
 'cats_vs_dogs',
 'celeb_a',
 'celeb_a_hq',
 'cfq',
 'chexpert',
 'cifar10',
 'cifar100',
 'cifar10_1',
 'cifar10_corrupted',
 'citrus_leaves',
 'cityscapes',
 'civil_comments',
 'clevr',
 'cmaterdb',
 'cnn_dailymail',
 'coco',
 'coil100',
 'colorectal_histology',
 'colorectal_histology_large',
 'cos_e',
 'curated_breast_imaging_ddsm',
 'cycle_gan',
 'deep_weeds',
 'definite_pronoun_resolution',
 'diabetic_retinopathy_detection',
 'div2k',
 'dmlab',
 'downsampled_imagenet',
 'dsprites',
 'dtd',
 'duke_ultrasound',
 'dummy_dataset_shared_generator',
 'dummy_mnist',
 'emnist',
 'eraser_multi_rc',
 'esnli',
 'eurosat',
 'fashion_mnist',
 'flic',
 'flores',
 'food101',
 'gap',
 'gigaword',
 'glue',
 'gr

## Load IMDB Reviews Dataset

In [4]:
tfds.load(name='imdb_reviews')

{'test': <DatasetV1Adapter shapes: {label: (), text: ()}, types: {label: tf.int64, text: tf.string}>,
 'train': <DatasetV1Adapter shapes: {label: (), text: ()}, types: {label: tf.int64, text: tf.string}>,
 'unsupervised': <DatasetV1Adapter shapes: {label: (), text: ()}, types: {label: tf.int64, text: tf.string}>}

## Convert IMDB Dataset into Test Train and Validation Dataset

In [0]:
train_data, validation_data, test_data = tfds.load(name="imdb_reviews",split=('train[:60%]', 'train[60%:]', 'test'), as_supervised=True)

In [6]:
type(train_data)

tensorflow.python.data.ops.dataset_ops.DatasetV1Adapter

## Lets see some Movie Reviews

In [7]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [8]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

## Use Word2Vec to convert these sentences into a Vector of Numbers using a Transfer Learning Technique

### We are using Neural Network inside Neural Network one for the IMDB Dataset and another for Word2Vec 

#### TensorFlow Hub is a  library for publication , discovery and consumption and resuables parts of Machine Learning models. Tf Hub supports transfer learning

In [0]:
pretrained_model = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(pretrained_model, input_shape=[], dtype=tf.string, trainable=True)

## Converting Movie Reviews into Embeding Vector

In [10]:
train_examples_batch[0:2]

<tf.Tensor: shape=(2,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell a

In [11]:
hub_layer(train_examples_batch[0:2])

<tf.Tensor: shape=(2, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ]],
      dtype=float32)>

## Creating Neural Network 

In [12]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16,activation="relu"))
model.add(tf.keras.layers.Dense(1,activation="sigmoid"))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

## Train model

In [14]:
model.fit(train_data.shuffle(10000).batch(512),
         epochs=20,
         validation_data=validation_data.batch(512),
         verbose=1)

Epoch 1/20
30/30 [==============================] - 3s 106ms/step - loss: 0.8227 - accuracy: 0.5338 - val_loss: 0.7137 - val_accuracy: 0.5807
Epoch 2/20
30/30 [==============================] - 3s 104ms/step - loss: 0.6750 - accuracy: 0.6089 - val_loss: 0.6421 - val_accuracy: 0.6400
Epoch 3/20
30/30 [==============================] - 3s 104ms/step - loss: 0.6149 - accuracy: 0.6645 - val_loss: 0.5988 - val_accuracy: 0.6838
Epoch 4/20
30/30 [==============================] - 3s 102ms/step - loss: 0.5715 - accuracy: 0.7027 - val_loss: 0.5653 - val_accuracy: 0.7131
Epoch 5/20
30/30 [==============================] - 3s 105ms/step - loss: 0.5366 - accuracy: 0.7353 - val_loss: 0.5344 - val_accuracy: 0.7416
Epoch 6/20
30/30 [==============================] - 3s 105ms/step - loss: 0.5013 - accuracy: 0.7659 - val_loss: 0.5055 - val_accuracy: 0.7635
Epoch 7/20
30/30 [==============================] - 3s 103ms/step - loss: 0.4702 - accuracy: 0.7887 - val_loss: 0.4798 - val_accuracy: 0.7799
Epoch 

In [15]:
model.predict(["This is the worst movie I have ever seen",
              "An excellent movie that I enjoyed a lot",
              "how can one make such a horrible movie? there is no story, acting direction everything is very poor",
               "I love this movie"])

array([[0.14506486],
       [0.9786699 ],
       [0.01249405],
       [0.87103856]], dtype=float32)

In [16]:
model.predict(["An excellent movie that I enjoyed a lot"])

array([[0.9786699]], dtype=float32)